In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from scipy.linalg import eigh

# ===========================
# Load and preprocess dataset
# ===========================

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

def load_schaefer_graphs(
    keep_fraction=0.3,
    outcome_col='Imp20PercentBPRS'
):
    # ===========================
    # Hardcoded file names
    # ===========================
    graph_file = "/content/drive/Shared drives/GNN/SchaeferAtlas_Rest_Results_Freq_008to09_wholebrain.xlsx"
    outcome_file = "/content/drive/Shared drives/GNN/RestingStateDataforAlex_3Networks.xlsx"

    demo_cols = ['Age', 'handedness', 'sex', 'PrimaryEthnicity',
                 'PrimaryRace', 'Education', 'Parental Education']

    # ===========================
    # Load outcome + demographics
    # ===========================
    df_labels = pd.read_excel(outcome_file, sheet_name='outcomeanddemographics', skiprows=1)
    demo_dict = df_labels.set_index('SID')[demo_cols].to_dict(orient='index')

    # Helper: threshold adjacency
    def threshold_graph(A, keep_fraction):
        triu_indices = np.triu_indices_from(A, k=1)
        edge_values = A[triu_indices]
        edge_values = edge_values[edge_values > 0]  # ignore zeros
        if len(edge_values) == 0:
            return np.zeros_like(A)
        threshold = np.percentile(edge_values, 100 * (1 - keep_fraction))
        mask = A >= threshold
        A_thresh = A * mask
        A_thresh = np.maximum(A_thresh, A_thresh.T)  # enforce symmetry
        return A_thresh

    # ===========================
    # Load only needed sheets
    # ===========================
    xl = pd.ExcelFile(graph_file)
    available_sheets = set(xl.sheet_names)

    # Get SIDs that actually have outcomes
    sids_with_outcome = df_labels['SID'].dropna().unique()
    sids_to_load = [sid for sid in sids_with_outcome if sid in available_sheets]

    graphs_by_sid = {}

    for sid in sids_to_load:
        try:
            # Load matrix for this patient
            mat_df = pd.read_excel(graph_file, sheet_name=sid, index_col=0)
            mat = mat_df.values.astype(float)

            mat = mat_df.values.astype(float)

            # Replace NaNs and infs with 0
            mat = np.nan_to_num(mat, nan=0.0, posinf=0.0, neginf=0.0)
            rel = mat
            # Take absolute value (since matrices can have negatives)
            mat = np.abs(mat)

            # Take absolute values (since matrices aren't absolute in this dataset)


            # Get outcome + demographics
            row = df_labels.loc[df_labels['SID'] == sid]
            outcome = row[outcome_col].values[0]
            demos = demo_dict.get(sid, None)

            # Threshold adjacency
            mat_thresh = threshold_graph(mat, keep_fraction)
            rel_thresh = threshold_graph(rel, keep_fraction)

            graphs_by_sid[sid] = {
                'abs': mat_thresh,      # absolute-valued adjacency
                'rel': rel_thresh,
                'demo': demos,
                'outcome': outcome,
                'nodes': list(mat_df.index)  # preserve brain region labels
            }
        except Exception as e:
            print(f"Skipping SID {sid} due to error: {e}")
            continue

    return graphs_by_sid


In [ ]:
#combines all the previous dataloading

import pandas as pd
import numpy as np



from itertools import product
import numpy as np


def load_and_preprocess_graphs(
    file_path,
    abs_keep_fraction=0.3,
    rel_keep_fraction=0.3,
    demo_cols=None,
    outcome_col='Imp20PercentBPRS'
):
    if demo_cols is None:
        demo_cols = ['Age', 'handedness', 'sex', 'PrimaryEthnicity', 'PrimaryRace', 'Education', 'Parental Education']

    # ===========================
    # Load dataset
    # ===========================
    df = pd.read_excel(file_path)
    df_labels = pd.read_excel(file_path, sheet_name='outcomeanddemographics', skiprows=1)

    # ===========================
    # Extract nodes and edges
    # ===========================
    edge_columns = df.columns.drop('SID')

    # Get all unique nodes
    nodes_set = set()
    for col in edge_columns:
        raw_node1, raw_node2 = col.split('-')
        node1 = raw_node1.replace('ABS_', '')
        node2 = raw_node2.replace('ABS_', '')
        nodes_set.add(node1)
        nodes_set.add(node2)

    nodes = sorted(list(nodes_set))
    n_nodes = len(nodes)
    node_to_idx = {node: i for i, node in enumerate(nodes)}

    abs_edge_cols, rel_edge_cols = [], []
    abs_edge_to_idx, rel_edge_to_idx = [], []

    for col in edge_columns:
        raw_node1, raw_node2 = col.split('-')
        node1 = raw_node1.replace('ABS_', '')
        node2 = raw_node2.replace('ABS_', '')
        idx1, idx2 = node_to_idx[node1], node_to_idx[node2]

        if 'ABS_' in col:
            abs_edge_cols.append(col)
            abs_edge_to_idx.append((idx1, idx2))
        else:
            rel_edge_cols.append(col)
            rel_edge_to_idx.append((idx1, idx2))

    # ===========================
    # Build adjacency matrices
    # ===========================
    abs_adj_matrices, rel_adj_matrices = [], []

    for _, row in df.iterrows():
        abs_adj = np.zeros((n_nodes, n_nodes))
        rel_adj = np.zeros((n_nodes, n_nodes))

        for col_idx, (i1, i2) in enumerate(abs_edge_to_idx):
            val = row[abs_edge_cols[col_idx]]
            abs_adj[i1, i2] = val
            abs_adj[i2, i1] = val

        for col_idx, (i1, i2) in enumerate(rel_edge_to_idx):
            val = row[rel_edge_cols[col_idx]]
            rel_adj[i1, i2] = val
            rel_adj[i2, i1] = val

        abs_adj_matrices.append(abs_adj)
        rel_adj_matrices.append(rel_adj)

    # ===========================
    # Thresholding function
    # ===========================
    def threshold_graph(A, keep_fraction):
        triu_indices = np.triu_indices_from(A, k=1)
        edge_values = A[triu_indices]
        threshold = np.percentile(edge_values, 100 * (1 - keep_fraction))
        mask = A >= threshold
        A_thresh = A * mask
        A_thresh = np.maximum(A_thresh, A_thresh.T)
        return A_thresh

    # ===========================
    # Build final dictionary
    # ===========================
    demo_dict = df_labels.set_index('SID')[demo_cols].to_dict(orient='index')

    graphs_by_sid = {}
    for sid, abs_mat, rel_mat in zip(df['SID'], abs_adj_matrices, rel_adj_matrices):
        demos = demo_dict.get(sid, None)
        abs_mat_thresh = threshold_graph(abs_mat, abs_keep_fraction)
        rel_mat_thresh = threshold_graph(rel_mat, rel_keep_fraction)
        outcome = df_labels.loc[df_labels['SID'] == sid, outcome_col].values[0] \
                  if sid in df_labels['SID'].values else None

        graphs_by_sid[sid] = {
            'abs': abs_mat_thresh,
            'rel': rel_mat_thresh,
            'demo': demos,
            'outcome': outcome
        }

    # Filter out subjects without outcome
    graphs_by_sid = {sid: g for sid, g in graphs_by_sid.items() if g['outcome'] is not None}

    return graphs_by_sid


In [ ]:
def inter_mixup(X_train, y_train, n_samples, alpha=0.2, hard_labels=False):
    """
    Generates n_samples synthetic examples using mixup.
    """
    X_new = []
    y_new = []

    for _ in range(n_samples):
        i, j = np.random.choice(len(X_train), size=2, replace=False)
        lam = np.random.beta(alpha, alpha)
        x_mix = lam * X_train[i] + (1 - lam) * X_train[j]
        y_mix = lam * y_train[i] + (1 - lam) * y_train[j]

        if hard_labels:
            y_mix = int(round(y_mix))  # convert to 0 or 1

        X_new.append(x_mix)
        y_new.append(y_mix)

    return np.vstack(X_new), np.array(y_new)


def intra_mixup(X_train, y_train, n_samples_per_class, alpha=0.2):
    """
    Generates synthetic samples by mixing features within the same class.
    X_train: (n_samples, n_features)
    y_train: (n_samples,)
    n_samples_per_class: number of synthetic samples to generate per class
    alpha: Beta distribution parameter
    """
    X_new, y_new = [], []
    classes = np.unique(y_train)

    for c in classes:
        idx_class = np.where(y_train == c)[0]
        for _ in range(n_samples_per_class):
            i, j = np.random.choice(idx_class, size=2, replace=False)
            lam = np.random.beta(alpha, alpha)
            x_mix = lam * X_train[i] + (1 - lam) * X_train[j]
            X_new.append(x_mix)
            y_new.append(c)  # hard label

    return np.vstack(X_new), np.array(y_new)


import networkx as nx
import numpy as np

def compute_more_graph_features(adj_matrix):
    """
    Computes a variety of node- and graph-level features for a weighted adjacency matrix.
    Returns a 1D feature vector.
    """
    G = nx.from_numpy_array(adj_matrix)
    features = []

    # ===========================
    # Node-level metrics (mean + std)
    # ===========================
    # Node strength (weighted degree)
    strength = np.array([s for n, s in G.degree(weight='weight')])
    features.extend([strength.mean(), strength.std()])


    # Clustering coefficient
    clustering = np.array(list(nx.clustering(G, weight='weight').values()))
    features.extend([clustering.mean(), clustering.std()])

    # Betweenness centrality
    try:
        bc = np.array(list(nx.betweenness_centrality(G, weight='weight').values()))
        features.extend([bc.mean(), bc.std()])
    except:
        features.extend([np.nan, np.nan])

    # Eigenvector centrality
    try:
        ec = np.array(list(nx.eigenvector_centrality(G, weight='weight', max_iter=500).values()))
        features.extend([ec.mean(), ec.std()])
    except:
        features.extend([np.nan, np.nan])

    # PageRank
    try:
        pr = np.array(list(nx.pagerank(G, weight='weight').values()))
        features.extend([pr.mean(), pr.std()])
    except:
        features.extend([np.nan, np.nan])

    # ===========================
    # Global graph metrics
    # ===========================
    # Global efficiency
    try:
        features.append(nx.global_efficiency(G))
    except:
        features.append(np.nan)

    # Average clustering
    try:
        features.append(nx.average_clustering(G, weight='weight'))
    except:
        features.append(np.nan)

    # Transitivity
    try:
        features.append(nx.transitivity(G))
    except:
        features.append(np.nan)

    # Assortativity (degree)
    try:
        features.append(nx.degree_assortativity_coefficient(G, weight='weight'))
    except:
        features.append(np.nan)

    # Density
    try:
        features.append(nx.density(G))
    except:
        features.append(np.nan)

    return np.array(features)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import time


from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor # Corrected import
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve
)

def add_noise(X, sigma=0.01):
    return X + np.random.normal(0, sigma, X.shape)

# ===========================
# 1. Graph feature computation
# ===========================
def compute_graph_features(adj_matrix, feature_flags):
    """
    Computes node- and graph-level features for a weighted adjacency matrix.
    Returns a 1D feature vector and corresponding feature names.
    """
    G = nx.from_numpy_array(adj_matrix)
    features = []
    feature_names = []

    # Node-level metrics
    strength = np.array([s for n, s in G.degree(weight='weight')])
    clustering_dict = nx.clustering(G, weight='weight')
    clustering = np.array(list(clustering_dict.values()))

    if feature_flags.get("strength_mean", True):
        features.append(strength.mean())
        feature_names.append("strength_mean")
    if feature_flags.get("strength_std", True):
        features.append(strength.std())
        feature_names.append("strength_std")
    if feature_flags.get("clustering_mean", True):
        features.append(clustering.mean())
        feature_names.append("clustering_mean")
    if feature_flags.get("clustering_std", True):
        features.append(clustering.std())
        feature_names.append("clustering_std")

    if feature_flags.get("betweenness_centrality", True):
        try:
            bc = np.array(list(nx.betweenness_centrality(G, weight='weight').values()))
            features.extend([bc.mean(), bc.std()])
            feature_names.extend(["betweenness_centrality_mean", "betweenness_centrality_std"])
        except:
            features.extend([np.nan, np.nan])
            feature_names.extend(["betweenness_centrality_mean", "betweenness_centrality_std"])


    if feature_flags.get("eigenvector_centrality", True):
        try:
            ec = np.array(list(nx.eigenvector_centrality(G, weight='weight', max_iter=500).values()))
            features.extend([ec.mean(), ec.std()])
            feature_names.extend(["eigenvector_centrality_mean", "eigenvector_centrality_std"])
        except:
            features.extend([np.nan, np.nan])
            feature_names.extend(["eigenvector_centrality_mean", "eigenvector_centrality_std"])


    if feature_flags.get("avg_clustering", True):
        try:
            features.append(nx.average_clustering(G, weight='weight'))
            feature_names.append("avg_clustering")
        except:
            features.append(np.nan)
            feature_names.append("avg_clustering")

    return features, feature_names

# ===========================
# 2. Load data and build feature matrix
# ===========================
file_path = '/content/drive/Shared drives/GNN/RestingStateDataforAlex_3Networks.xlsx'
graphs_by_sid = load_and_preprocess_graphs(file_path, abs_keep_fraction=1.0, rel_keep_fraction=1.0)
graphs_by_sid = load_schaefer_graphs()

# ---------------------------
# Feature flags (toggle True/False)
# ---------------------------
feature_flags = {
    "strength_mean": True,
    "strength_std": True,
    "clustering_mean": True,
    "clustering_std": True,
    "betweenness_centrality": True, # Added betweenness centrality flag
    "eigenvector_centrality": True, # Added eigenvector centrality flag
    "avg_clustering": True
}


df = pd.read_excel(file_path, sheet_name='outcomeanddemographics', skiprows=1)
bprs_map = dict(zip(df['SID'], df['BPRS_Baseline']))
print(df.columns)


feature_list = []
all_feature_names = None
for sid in graphs_by_sid.keys():
    features, current_feature_names = compute_graph_features(graphs_by_sid[sid]['abs'], feature_flags)

    # Convert list to dictionary for easier handling
    features_dict = dict(zip(current_feature_names, features))

    # Add BPRS_Baseline
    features_dict['BPRS_Baseline'] = bprs_map.get(sid, np.nan)

    feature_list.append(features_dict)

# Convert list of dictionaries to DataFrame
all_features_df = pd.DataFrame(feature_list)

X = all_features_df.to_numpy()  # shape: (num_sids, num_features)
y = np.array([graphs_by_sid[sid]['outcome'] for sid in graphs_by_sid.keys()])

# Remove NaN columns and corresponding feature names
nan_cols = np.isnan(X).any(axis=0)
X = X[:, ~nan_cols]
##feature_names = [all_feature_names[i] for i in range(len(all_feature_names)) if not nan_cols[i]]

print("X shape after NaN removal:", X.shape)
print("feature_flags:", feature_flags)
#print("feature_names:", feature_names)


print((y == 1).sum())  # number of elements equal to 1
print((y == 0).sum())  # number of elements equal to 0


# ===========================
# 3. 20-run 5-fold CV
# ===========================

# The prefix B means baseline
#clf = make_pipeline(StandardScaler(), SGDRegressor(max_iter=500, tol=1e-3))
#Bclf = make_pipeline(StandardScaler(), SGDRegressor(max_iter=500, tol=1e-3))
#clf = MLPRegressor(hidden_layer_sizes=(100,100), activation='relu', solver='adam', max_iter=500)
#Bclf = MLPRegressor(hidden_layer_sizes=(100,100), activation='relu', solver='adam', max_iter=500)
clf = SVR(kernel='rbf')
Bclf = SVR(kernel='rbf')


n_runs = 100  # number of repeated CV runs
k_folds = 10

all_accs, all_precisions, all_recalls, all_f1s, all_aucs = [], [], [], [], []
all_coefs = []
Ball_accs, Ball_precisions, Ball_recalls, Ball_f1s, Ball_aucs = [], [], [], [], []
Ball_coefs = []

random_seed = int(time.time()) # or np.random.randint(0, 10000)
for run in range(n_runs):
    print(f"=== Run {run+1}/{n_runs} ===")


    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=random_seed + run)

    accs, precisions, recalls, f1s, aucs, coefs = [], [], [], [], [], []
    Baccs, Bprecisions, Brecalls, Bf1s, Baucs, Bcoefs = [], [], [], [], [], []


    for train_idx, test_idx in skf.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]

        # ---------------------------
        # Mixup (optional)
        # ---------------------------
#       X_aug, y_aug = intra_mixup(X_train, y_train, n_samples_per_class=40, alpha=0.9) #same class mixup
        X_aug, y_aug = inter_mixup(X_train, y_train, n_samples=70, alpha=0.7, hard_labels=False)
        X_train_aug = np.vstack([X_train, X_aug])
        y_train_aug = np.concatenate([y_train, y_aug])

#        print("X shape before Mixup:", X_train.shape)
#        print("X shape after Mixup:", X_train_aug.shape)

        #X_train_aug, y_train_aug = X_train, y_train  # no mixup
        #X_train_aug = add_noise(X_train_aug)

        clf.fit(X_train_aug, y_train_aug)
        Bclf.fit(X_train, y_train)

        y_prob = clf.predict(X[test_idx])
#        y_prob = clf.predict_proba(X[test_idx])[:,1]

        y_prob = np.clip(y_prob, 0, 1)  # ensure in [0,1]
        # Threshold for discrete labels
        y_pred = (y_prob >= 0.5).astype(int)


        By_prob = Bclf.predict(X[test_idx])
#        By_prob = Bclf.predict_proba(X[test_idx])[:,1]
        By_prob = np.clip(By_prob, 0, 1)  # ensure in [0,1]
        # Threshold for discrete labels
        By_pred = (By_prob >= 0.5).astype(int)

        #print(np.unique(y_pred, return_counts=True))

        accs.append(accuracy_score(y[test_idx], y_pred))
        precisions.append(precision_score(y[test_idx], y_pred,zero_division=0))
        recalls.append(recall_score(y[test_idx], y_pred,zero_division=0))
        f1s.append(f1_score(y[test_idx], y_pred,zero_division=0))
 #       aucs.append(roc_auc_score(y[test_idx], y_prob))

        Baccs.append(accuracy_score(y[test_idx], By_pred))
        Bprecisions.append(precision_score(y[test_idx], By_pred,zero_division=0))
        Brecalls.append(recall_score(y[test_idx], By_pred,zero_division=0))
        Bf1s.append(f1_score(y[test_idx], By_pred,zero_division=0))
#        Baucs.append(roc_auc_score(y[test_idx], By_prob))

#        log_reg = clf.named_steps['logisticregression']
#        coefs.append(log_reg.coef_[0])

#        Blog_reg = Bclf.named_steps['logisticregression']
#        Bcoefs.append(log_reg.coef_[0])


    # Append this run's results to overall
    all_accs.extend(accs)
    all_precisions.extend(precisions)
    all_recalls.extend(recalls)
    all_f1s.extend(f1s)
    all_aucs.extend(aucs)
  #  all_coefs.append(np.mean(coefs, axis=0))

    Ball_accs.extend(Baccs)
    Ball_precisions.extend(Bprecisions)
    Ball_recalls.extend(Brecalls)
    Ball_f1s.extend(Bf1s)
    Ball_aucs.extend(Baucs)
   # Ball_coefs.append(np.mean(Bcoefs, axis=0))

    print(f"Accuracy Baseline first : {np.mean(Baccs):.2f}\t{np.mean(accs):.2f}")
# ===========================
# 4. Report overall metrics
# ===========================
print("\n=== Overall : {n_runs}-run {k_folds}-fold CV metrics === Baseline is First Then MixUp")
print(f"Accuracy  : {np.mean(Ball_accs):.3f}\t{np.mean(all_accs):.3f}")
print(f"Precision : {np.mean(Ball_precisions):.3f}\t{np.mean(all_precisions):.3f}")
print(f"Recall    : {np.mean(Ball_recalls):.3f}\t{np.mean(all_recalls):.3f}")
print(f"F1-score  : {np.mean(Ball_f1s):.3f}\t{np.mean(all_f1s):.3f}")
print(f"AUC       : {np.mean(Ball_aucs):.3f}\t{np.mean(all_aucs):.3f}")

# ===========================
# 5. Feature importance
# ===========================
#mean_coef = np.mean(all_coefs, axis=0)
#importance = np.abs(mean_coef)

#print("\nFeature importance (avg |coef| across runs):")
#for name, imp in zip(feature_names, importance):
#    print(f"{name:<20}: {imp:.3f}")

Index(['SID', 'Imp20PercentBPRS', 'Age', 'handedness', 'sex',
       'PrimaryEthnicity', 'PrimaryRace', 'Unnamed: 7', 'Education',
       'Parental Education', 'Unnamed: 10', 'BPRS_Baseline'],
      dtype='object')
X shape after NaN removal: (56, 10)
feature_flags: {'strength_mean': True, 'strength_std': True, 'clustering_mean': True, 'clustering_std': True, 'betweenness_centrality': True, 'eigenvector_centrality': True, 'avg_clustering': True}
29
27
=== Run 1/100 ===
Accuracy Baseline first : 0.68	0.63
=== Run 2/100 ===
Accuracy Baseline first : 0.66	0.63
=== Run 3/100 ===
Accuracy Baseline first : 0.65	0.63
=== Run 4/100 ===
Accuracy Baseline first : 0.69	0.63
=== Run 5/100 ===
Accuracy Baseline first : 0.59	0.61
=== Run 6/100 ===
Accuracy Baseline first : 0.62	0.68
=== Run 7/100 ===
Accuracy Baseline first : 0.71	0.62
=== Run 8/100 ===
Accuracy Baseline first : 0.67	0.69
=== Run 9/100 ===
Accuracy Baseline first : 0.68	0.68
=== Run 10/100 ===
Accuracy Baseline first : 0.68	0.62
=== 

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
#clf = make_pipeline(StandardScaler(), SGDRegressor(max_iter=500, tol=1e-3))
#Bclf = make_pipeline(StandardScaler(), SGDRegressor(max_iter=500, tol=1e-3))
clf = MLPRegressor(hidden_layer_sizes=(100,100), activation='relu', solver='adam', max_iter=500)
Bclf = MLPRegressor(hidden_layer_sizes=(100,100), activation='relu', solver='adam', max_iter=500)
#clf = SVR(kernel='rbf')
#Bclf = SVR(kernel='rbf')


n_runs = 100  # number of repeated CV runs
k_folds = 10

all_accs, all_precisions, all_recalls, all_f1s, all_aucs = [], [], [], [], []
all_coefs = []
Ball_accs, Ball_precisions, Ball_recalls, Ball_f1s, Ball_aucs = [], [], [], [], []
Ball_coefs = []

random_seed = int(time.time()) # or np.random.randint(0, 10000)
for run in range(n_runs):
    print(f"=== Run {run+1}/{n_runs} ===")


    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=random_seed + run)

    accs, precisions, recalls, f1s, aucs, coefs = [], [], [], [], [], []
    Baccs, Bprecisions, Brecalls, Bf1s, Baucs, Bcoefs = [], [], [], [], [], []


    for train_idx, test_idx in skf.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]

        # ---------------------------
        # Mixup (optional)
        # ---------------------------
#       X_aug, y_aug = intra_mixup(X_train, y_train, n_samples_per_class=40, alpha=0.9) #same class mixup
        X_aug, y_aug = inter_mixup(X_train, y_train, n_samples=70, alpha=0.7, hard_labels=False)
        X_train_aug = np.vstack([X_train, X_aug])
        y_train_aug = np.concatenate([y_train, y_aug])

#        print("X shape before Mixup:", X_train.shape)
#        print("X shape after Mixup:", X_train_aug.shape)

        #X_train_aug, y_train_aug = X_train, y_train  # no mixup
        #X_train_aug = add_noise(X_train_aug)

        clf.fit(X_train_aug, y_train_aug)
        Bclf.fit(X_train, y_train)

        y_prob = clf.predict(X[test_idx])
#        y_prob = clf.predict_proba(X[test_idx])[:,1]

        y_prob = np.clip(y_prob, 0, 1)  # ensure in [0,1]
        # Threshold for discrete labels
        y_pred = (y_prob >= 0.5).astype(int)


        By_prob = Bclf.predict(X[test_idx])
#        By_prob = Bclf.predict_proba(X[test_idx])[:,1]
        By_prob = np.clip(By_prob, 0, 1)  # ensure in [0,1]
        # Threshold for discrete labels
        By_pred = (By_prob >= 0.5).astype(int)

        #print(np.unique(y_pred, return_counts=True))

        accs.append(accuracy_score(y[test_idx], y_pred))
        precisions.append(precision_score(y[test_idx], y_pred,zero_division=0))
        recalls.append(recall_score(y[test_idx], y_pred,zero_division=0))
        f1s.append(f1_score(y[test_idx], y_pred,zero_division=0))
 #       aucs.append(roc_auc_score(y[test_idx], y_prob))

        Baccs.append(accuracy_score(y[test_idx], By_pred))
        Bprecisions.append(precision_score(y[test_idx], By_pred,zero_division=0))
        Brecalls.append(recall_score(y[test_idx], By_pred,zero_division=0))
        Bf1s.append(f1_score(y[test_idx], By_pred,zero_division=0))
#        Baucs.append(roc_auc_score(y[test_idx], By_prob))

#        log_reg = clf.named_steps['logisticregression']
#        coefs.append(log_reg.coef_[0])

#        Blog_reg = Bclf.named_steps['logisticregression']
#        Bcoefs.append(log_reg.coef_[0])


    # Append this run's results to overall
    all_accs.extend(accs)
    all_precisions.extend(precisions)
    all_recalls.extend(recalls)
    all_f1s.extend(f1s)
    all_aucs.extend(aucs)
  #  all_coefs.append(np.mean(coefs, axis=0))

    Ball_accs.extend(Baccs)
    Ball_precisions.extend(Bprecisions)
    Ball_recalls.extend(Brecalls)
    Ball_f1s.extend(Bf1s)
    Ball_aucs.extend(Baucs)
   # Ball_coefs.append(np.mean(Bcoefs, axis=0))

    print(f"Accuracy Baseline first : {np.mean(Baccs):.2f}\t{np.mean(accs):.2f}")
# ===========================
# 4. Report overall metrics
# ===========================
print("\n=== Overall : {n_runs}-run {k_folds}-fold CV metrics === Baseline is First Then MixUp")
print(f"Accuracy  : {np.mean(Ball_accs):.3f}\t{np.mean(all_accs):.3f}")
print(f"Precision : {np.mean(Ball_precisions):.3f}\t{np.mean(all_precisions):.3f}")
print(f"Recall    : {np.mean(Ball_recalls):.3f}\t{np.mean(all_recalls):.3f}")
print(f"F1-score  : {np.mean(Ball_f1s):.3f}\t{np.mean(all_f1s):.3f}")
print(f"AUC       : {np.mean(Ball_aucs):.3f}\t{np.mean(all_aucs):.3f}")

# ===========================
# 5. Feature importance
# ===========================
#mean_coef = np.mean(all_coefs, axis=0)
#importance = np.abs(mean_coef)

#print("\nFeature importance (avg |coef| across runs):")
#for name, imp in zip(feature_names, importance):
#    print(f"{name:<20}: {imp:.3f}")

=== Run 1/100 ===
Accuracy Baseline first : 0.65	0.66
=== Run 2/100 ===
Accuracy Baseline first : 0.70	0.57
=== Run 3/100 ===
Accuracy Baseline first : 0.63	0.63
=== Run 4/100 ===
Accuracy Baseline first : 0.62	0.66
=== Run 5/100 ===
Accuracy Baseline first : 0.59	0.59
=== Run 6/100 ===
Accuracy Baseline first : 0.65	0.65
=== Run 7/100 ===
Accuracy Baseline first : 0.66	0.63
=== Run 8/100 ===
Accuracy Baseline first : 0.62	0.56
=== Run 9/100 ===
Accuracy Baseline first : 0.63	0.67
=== Run 10/100 ===
Accuracy Baseline first : 0.71	0.58
=== Run 11/100 ===
Accuracy Baseline first : 0.68	0.64
=== Run 12/100 ===
Accuracy Baseline first : 0.66	0.64
=== Run 13/100 ===
Accuracy Baseline first : 0.60	0.63
=== Run 14/100 ===
Accuracy Baseline first : 0.70	0.56
=== Run 15/100 ===
Accuracy Baseline first : 0.64	0.59
=== Run 16/100 ===
Accuracy Baseline first : 0.53	0.59
=== Run 17/100 ===
Accuracy Baseline first : 0.64	0.67
=== Run 18/100 ===
Accuracy Baseline first : 0.64	0.61
=== Run 19/100 ===


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy Baseline first : 0.63	0.68
=== Run 46/100 ===
Accuracy Baseline first : 0.77	0.65
=== Run 47/100 ===
Accuracy Baseline first : 0.65	0.63
=== Run 48/100 ===
Accuracy Baseline first : 0.56	0.55
=== Run 49/100 ===
Accuracy Baseline first : 0.73	0.71
=== Run 50/100 ===
Accuracy Baseline first : 0.63	0.57
=== Run 51/100 ===
Accuracy Baseline first : 0.71	0.63
=== Run 52/100 ===
Accuracy Baseline first : 0.63	0.57
=== Run 53/100 ===
Accuracy Baseline first : 0.69	0.65
=== Run 54/100 ===
Accuracy Baseline first : 0.65	0.59
=== Run 55/100 ===
Accuracy Baseline first : 0.68	0.60
=== Run 56/100 ===
Accuracy Baseline first : 0.64	0.66
=== Run 57/100 ===
Accuracy Baseline first : 0.71	0.61
=== Run 58/100 ===
Accuracy Baseline first : 0.63	0.60
=== Run 59/100 ===
Accuracy Baseline first : 0.64	0.61
=== Run 60/100 ===
Accuracy Baseline first : 0.64	0.72
=== Run 61/100 ===
Accuracy Baseline first : 0.68	0.50
=== Run 62/100 ===
Accuracy Baseline first : 0.61	0.52
=== Run 63/100 ===
Accuracy B

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy Baseline first : 0.59	0.64
=== Run 68/100 ===
Accuracy Baseline first : 0.61	0.75
=== Run 69/100 ===
Accuracy Baseline first : 0.63	0.72
=== Run 70/100 ===
Accuracy Baseline first : 0.66	0.66
=== Run 71/100 ===
Accuracy Baseline first : 0.72	0.69
=== Run 72/100 ===
Accuracy Baseline first : 0.65	0.70
=== Run 73/100 ===


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy Baseline first : 0.60	0.65
=== Run 74/100 ===
Accuracy Baseline first : 0.64	0.60
=== Run 75/100 ===
Accuracy Baseline first : 0.66	0.63
=== Run 76/100 ===
Accuracy Baseline first : 0.59	0.59
=== Run 77/100 ===
Accuracy Baseline first : 0.69	0.57
=== Run 78/100 ===
Accuracy Baseline first : 0.61	0.64
=== Run 79/100 ===
Accuracy Baseline first : 0.73	0.67
=== Run 80/100 ===
Accuracy Baseline first : 0.59	0.63
=== Run 81/100 ===
Accuracy Baseline first : 0.57	0.73
=== Run 82/100 ===
Accuracy Baseline first : 0.61	0.63
=== Run 83/100 ===


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy Baseline first : 0.61	0.62
=== Run 84/100 ===
Accuracy Baseline first : 0.75	0.62
=== Run 85/100 ===
Accuracy Baseline first : 0.62	0.67
=== Run 86/100 ===
Accuracy Baseline first : 0.70	0.66
=== Run 87/100 ===
Accuracy Baseline first : 0.64	0.75
=== Run 88/100 ===
Accuracy Baseline first : 0.66	0.62
=== Run 89/100 ===
Accuracy Baseline first : 0.62	0.68
=== Run 90/100 ===
Accuracy Baseline first : 0.70	0.58
=== Run 91/100 ===
Accuracy Baseline first : 0.70	0.64
=== Run 92/100 ===
Accuracy Baseline first : 0.63	0.64
=== Run 93/100 ===
Accuracy Baseline first : 0.61	0.70
=== Run 94/100 ===
Accuracy Baseline first : 0.64	0.70
=== Run 95/100 ===
Accuracy Baseline first : 0.66	0.67
=== Run 96/100 ===
Accuracy Baseline first : 0.61	0.67
=== Run 97/100 ===
Accuracy Baseline first : 0.60	0.65
=== Run 98/100 ===
Accuracy Baseline first : 0.56	0.65
=== Run 99/100 ===
Accuracy Baseline first : 0.63	0.55
=== Run 100/100 ===
Accuracy Baseline first : 0.70	0.56

=== Overall : {n_runs}-run 

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
